In [1]:
import gensim
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd

In [2]:
train_data = pd.read_csv("train.csv")
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True) 

In [14]:
description_list=train_data['Description']

In [45]:
def get_word2vec(description):
    description = description_list[0]
    description = description.lower()
    description = re.sub('[^a-zA-Z]', ' ', description )  
    description = re.sub(r'\s+', ' ', description)
    description_tokens = nltk.sent_tokenize(description)
    all_words = [nltk.word_tokenize(sent) for sent in description_tokens]
    for i in range(len(all_words)):  
        all_words[i] = [w for w in all_words[i] if w not in stopwords.words('english')]
    all_words = all_words[0]
    avg_word2vec_words = np.zeros(300)
    for word in all_words:
        avg_word2vec_words += model[word]
        
    return avg_word2vec_words
    

In [62]:
def convert_text_col_word2vec(column):
    col_vector_list = []
    for description in column:
        description_vector = get_word2vec(description)
        col_vector_list.append(description_vector)
    return col_vector_list

In [50]:
description_vector_list = np.array(description_vector_list)

In [57]:
from sklearn.neural_network import MLPClassifier

In [58]:
def _run_nn(train_feature_matrix, train_label_matrix, test_feature_matrix):
    nn_clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(50, 30), random_state=1)
    nn_train_feature_matrix = train_feature_matrix.astype(np.float64)
    nn_test_feature_matrix = test_feature_matrix.astype(np.float64)
    nn_clf.fit(nn_train_feature_matrix, train_label_matrix)
    nn_predictions = nn_clf.predict(nn_test_feature_matrix)
    return nn_predictions

In [59]:
def calculate_macro_f1_score(predictions, true_labels):
    true_positives = [0 for i in range(11)]
    false_positives = [0 for i in range(11)]
    false_negatives = [0 for i in range(11)]

    if len(predictions) != len(true_labels):
        print("bug in code, length of predictions should match length of true_labels")
        return None
    for i in range(len(predictions)):
        if predictions[i] == true_labels[i]:
            true_positives[predictions[i]] += 1
        else:
            false_positives[predictions[i]] += 1
            false_negatives[true_labels[i]] += 1

    total_classes = 0
    total_f1 = 0
    for i in range(11):
        if true_positives[i]==0 and false_positives[i]==0:
            continue
        elif true_positives[i]==0 and false_negatives[i]==0:
            continue
        prec = true_positives[i]*1.0/(true_positives[i] + false_positives[i])
        recall = true_positives[i]*1.0/(true_positives[i]+false_negatives[i])
        f1=0
        if prec+recall != 0:
            f1 = 2*prec*recall/(prec+recall)
            total_classes += 1
            total_f1 += f1
    return total_f1*100.0/total_classes

In [ ]:
name_vector_column = convert_text_col_word2vec(train_data['Name'])

In [ ]:
'''
Code for Artificial Neural Network
'''
start_time = time.time()
predicted_labels = _run_nn(train_feature_matrix, train_label_matrix, test_feature_matrix)
end_time = time.time() - start_time